# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: Polynomials

Michael Reefe

This example notebook provides a quick tutorial on how to use the additive and multiplicative polynomial components in LOKI.

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()

using Loki

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


Now we want to load in our data. For this example, we'll be using a far-ultraviolet spectrum of the Fornax Cluster BCG (NGC 1399) from FUSE. This spectrum has been background-subtracted, but the background is not perfect, so we will model residual background emission with an additive polynomial.  We'll also use multiplicative polynomials on the purely stellar continuum to account for any possible relative flux calibration offsets.

Since this is not a JWST formatted data product, just like with the SDSS example notebook, there is no built-in functionality for LOKI to read its contents and convert it into a data format that it can work with.  This is also notably not an IFU data cube, but just a single spectrum.  This is fine, as we can effectively treat it as an IFU cube with spatial dimensions of 1x1.  We'll start by reading in the FITS file with Julia's `FITSIO` module and storing relevant information in arrays.

In [2]:
using FITSIO                    # for reading/writing FITS files
using Unitful, UnitfulAstro     # to add units to the data

In [3]:
# Read in the FITS file and header
hdu = FITS("abells0373-g1.final.fits.gz")
hdr0 = read_header(hdu[1])
hdr = read_header(hdu["SPECTRUM"])

# Wavelength vector (angstrom)
λ = read(hdu["SPECTRUM"], "WAVE") .* u"angstrom"

# Flux - background (erg/s/cm^2/ang)
F = (read(hdu["SPECTRUM"], "FLUX") .- read(hdu["SPECTRUM"], "BKGD")) .* u"erg/s/cm^2/angstrom"
# Reshape to a 1 x 1 x (n_wavelength) cube
F = reshape(F, (1,1,length(F))) 

# Error cube (erg/s/cm^2/ang)
eF = read(hdu["SPECTRUM"], "ERROR") .* u"erg/s/cm^2/angstrom"
# Reshape to a 1 x 1 x (n_wavelength) cube
eF = reshape(eF, (1,1,length(eF)))

# Bad pixel mask 
mask = read(hdu["SPECTRUM"], "QUALITY") .!= 100.0
mask .|= .~isfinite.(F[1,1,:]) .| .~isfinite.(eF[1,1,:])
mask = reshape(mask, (1,1,length(mask)))

# Auxiliary information
z = 0.004753
ra = hdr0["RA_TARG"]
dec = hdr0["DEC_TARG"]

# Switches
rest_frame = false    # the input spectrum is not in the rest frame
masked = false        # the bad pixel mask has not been applied (i.e. bad pixels are still in the spectrum)
vacuum_wave = true    # the wavelengths provided are in vacuum wavelengths (SDSS spectra already come in vacuum wavelengths)
dereddened = false    # the spectrum has not been corrected for Milky Way dust absorption along the line of sight
log_binned = false    # the input wavelength vector is logarithmically spaced
sky_aligned = true    # this parameter only makes sense for a full IFU cube -- if true, the input cube has x/y axes aligned with
                      # the RA/Dec axes on the sky, otherwise, it may have any arbitrary orientation

# Since LOKI works in intensities rather than fluxes, we need the solid angle covered
# by each pixel in the input data cube.  Since we only have a single spectrum here,
# we take the solid angle as that which is covered by the FUSE aperture, which have a
# size of 30" x 30"
Ω = uconvert(u"sr", (30.0u"arcsecond")^2)

# Then we convert the fluxes into intensities
I = F ./ Ω
σ = eF ./ Ω

# We also need some measurements of the *spectral* and *spatial* resolutions of the input cube.

# Again, in our case since there is only 1 "pixel", we don't really have a concept of spatial resolution. So we'll
# just assume here that the spatial resolution is equal to the size of the aperture:
psf_fwhm = uconvert(u"arcsecond", sqrt(Ω))

# For the spectral resolution, FUSE nominally has a really high resolution of ~15 km/s or R ~ 20,000
R = 20000. .* ones(length(λ))

println("Bad pixels: $(sum(mask))/$(length(λ))")

# Close the FITS file
close(hdu)

Bad pixels: 72/1459


In [4]:
# Create a LOKI Cube object
cube = from_data(Ω, z, λ, I; α=ra, δ=dec, psf_fwhm=psf_fwhm, R=R, wcs=nothing, psf_model=nothing,
                 rest_frame=rest_frame, masked=masked, vacuum_wave=vacuum_wave, dereddened=dereddened, sky_aligned=sky_aligned, log_binned=log_binned)
# Create an Observation object
obs = from_cubes("abells0373", z, [cube], [0], inst="FUSE")

┌ Warning: The DataCube format is not JWST! Please manually input the channel edges (if any), otherwise it will be assumed that there is only one channel
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/cubedata.jl:635


Observation(Dict{Any, DataCube}(0 => DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[904.2899780273438 Å, 904.4849853515625 Å, 904.6799926757812 Å, 904.875 Å, 905.0700073242188 Å, 905.2650146484375 Å, 905.4600219726562 Å, 905.655029296875 Å, 905.8499755859375 Å, 906.0449829101562 Å  …  1186.6629638671875 Å, 1186.8580322265625 Å, 1187.052978515625 Å, 1187.248046875 Å, 1187.4429931640625 Å, 1187.637939453125 Å, 1187.8330078125 Å, 1188.0279541015625 Å, 1188.2230224609375 Å, 1188.41796875 Å], Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}[4.097816007256042e-7 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; -3.9421058412359453e-7 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; -2.2729045855902156e-7 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; … ;;; 1.542694483733691e-6 erg Å⁻¹ cm⁻² s⁻¹ sr⁻¹;;; 1.3852290677257685e-6

In [5]:
channel = 0  # (this is just used as a dummy value when the instrument doesn't have distinct spectral channels)
nm = "NGC1399"
run_name = "$(nm)_example"

"NGC1399_example"

Before fitting, we want to do some pre-processing on the data, just like in the MIRI example.  Exactly which pre-processing steps are performed are largely controlled by the boolean parameters listed above when creating the Cube object (i.e. `rest_frame`, `masked`, `vacuum_wave`, `dereddened`, `log_binned`, and `sky_aligned`). The corrections are actually carried out by calling the `correct!` function.  This is also where we could combine data from multiple channels into a single cube, if desired, using the `combine_channels!` function, but in this quick example we only have one sub-channel.

In [6]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)
    
    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end


new_wave contains values outside the range in old_wave, new_fluxes and new_errs will be filled with the value set in the 'fill' keyword argument. 

[ Info: Using SFD98 dust map at (α=54.622083°, δ=-35.450278°): E(B-V)=0.012660949453254057
┌ Warning: Input wavelength vector has values outside the allowable range of 1/λ > 11. Returning ones.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/math.jl:932
┌ Warning: Input wavelength vector has values outside the allowable range of 1/λ > 11. Returning ones.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/math.jl:932
┌ Warning: Input wavelength vector has values outside the allowable range of 1/λ > 11. Returning ones.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/math.jl:932
┌ Warning: Input wavelength vector has values outside the allowable range of 1/λ > 11. Returning ones.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/math.jl:932
┌ Warning: Input wavelength vector has values outsid

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data, same as with the MIRI example, but without an aperture argument since there is only one spaxel anyways.

\*\*\*IMPORTANT\*\*\*: Since this is a far-UV spectrum, we need to include young, hot stars in the stellar continuum, which are not included by default.  If you want your fit to look like my example plot below, you'll need to go into the options.toml configuration file, find the "[starts.teff]" option, and adjust to min=10000 K and max=250000 K.  I also adjusted "[stars.logg]" to min=-0.5 and max=+9.0.  If you want to speed up the fitting, you can try limiting the number of stellar templates by restricting "[stars.logz]" to min=-1.0 and max=-1.0, which shouldn't have a drastic effect on the final results.

In [ ]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
C_KMS = 299792.458

cube_fitter = CubeFitter(
    obs.channels[channel],
    obs.z, 
    run_name; 
    parallel=false, 
    plot_spaxels=:both, 
    plot_maps=false,  # <= this disables plotting 2D parameter maps, which we do here because it's a 1x1 cube and maps wouldn't really be informative
    save_fits=true,
    extinction_curve="calz", 
    # Here we set the degrees of the additive and multiplicative polynomials.
    # Higher degree = more freedom, but also more free parameters (i.e. longer fitting time) and can lead to over-fitting.
    # Note: apoly_degree must be at least 0 to include an additive polynomial 
    #       mpoly_degree must be at least 1 to include a multiplicative polynomial (as the 0th order term is disabled)
    apoly_degree=5,
    mpoly_degree=20,
    # Here we set the type of polynomials (the options are either Legendre or Chebyshev; I'll do one of each for comparison's sake)
    apoly_type="Legendre",
    mpoly_type="Chebyshev",
    # A few more various options
    linemask_width=100.0u"km/s",
    stellar_template_type="stars",
    ssp_regularize=0.,
    # Mask out some bad regions in the FUSE spectra 
    user_mask=[
        (1-500.0/C_KMS, 1+500.0/C_KMS).*972.5365.*u"angstrom" ./ (1+z), # Lyman-gamma airglow
        (1082.702, 1086.823).*u"angstrom" ./ (1+z),                           # FUSE chip gap
        (1-500.0/C_KMS, 1+500.0/C_KMS).*1025.722u"angstrom" ./ (1+z),   # Lyman-beta airglow
    ]
)
println(size(cube_fitter.ssps.templates))

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter)

[ Info: Preparing output directories
┌ Warning: The maximum wavelength in the input spectrum is < 2 μm! Thermal dust emission will be disabled.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/create_params.jl:363
┌ Warning: There are no PAH features within the input wavelength range of the spectrum. The use_pah_templates option will be disabled.
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/util/create_params.jl:540
[ Info: Generating 400 simple stellar population templates with ages ∈ (0.0010000000000000002 Gyr, 13.7 Gyr), log(Z/Zsun) ∈ (-2.3, 0.4)
[ Info: Resampling wavelengths...


Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.50 ms/it)


[ Info: Resampling ages/logzs...


Progress: 100%|███████████████████████████| Time: 0:00:00 ( 0.44 ms/it)


[ Info: Applying instrumental broadening...


Progress: 100%|███████████████████████████| Time: 0:00:00 ( 1.31 ms/it)


(1458, 400)
┌ Info: 
│ 
│ #############################################################################
│ ######## BEGINNING FULL CUBE FITTING ROUTINE FOR NGC1399_example ########
│ #############################################################################
│ 
│ ------------------------
│ Worker Processes:     1
│ Threads per process:  1
└ ------------------------
[ Info: ===> Preparing output data structures... <===
[ Info: Integrating spectrum across the whole cube...
[ Info: ===> Beginning integrated spectrum fitting... <===
[ Info: ===> Generating parameter maps and model cubes... <===
[ Info: ===> Writing FITS outputs... <===
[ Info: ===> Writing CSV tables... <=== 
┌ Info: 
│ 
│ #############################################################################
│ ################################### Done!! ##################################
└ #############################################################################


(CubeFitter{Float64, Int64, Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐋⁻¹ 𝐓⁻³, nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[900.0122199459404 Å, 900.2063047849193 Å, 900.4004314776906 Å, 900.594600033281 Å, 900.7888104607164 Å, 900.983062769027 Å, 901.1773569672444 Å, 901.3716930644019 Å, 901.566071069535 Å, 901.7604909916819 Å  …  1180.4640299919058 Å, 1180.718593392384 Å, 1180.9732116886687 Å, 1181.2278848925987 Å, 1181.4826130160131 Å, 1181.7373960707557 Å, 1181.9922340686726 Å, 1182.247127021612 Å, 1182.5020749414246 Å, 1182.7570778399652 Å], Quantity{Float64, 𝐌 𝐋⁻¹ 𝐓⁻³, Unitful.FreeUnits{(Å⁻¹, erg, cm⁻², s⁻¹, sr⁻

And the results can be found in the "output_[run_name]" directory!
I'm going to do a comparison.  First, here is what the fit looks like WITHOUT including any polynomials (setting apoly_degree=-1 and mpoly_degree=-1):

![](./Fornax.nopoly.png)

(Note: The O VI line is not that big. It's compensating for the poor continuum fit in that region)

And here is the fit WITH polynomials (apoly_degree=5 and mpoly_degree=20):

![](./Fornax.poly.png)

The orange line shows the final model.  The decomposed components of the model consist of:
- Stellar continuum, in pink
- Emission lines, in purple
- Additive polynomial, in dotted grey (bottom)
- Extinction, in dotted gray (top, read from the right axis)

There are a few things to notice here.  And this is meant to be just as much a cautionary tale on using polynomials as it is a showcase of their potential.  The polynomial fit does have a better overall reduced chi^2, and visually fits better at the ends of the spectrum.  But one will notice that the fit is actually worse in the middle (from ~1010-1040 angstroms).  Polynomials are very flexible, but they're not a fix-all by any means.  Also notice how the inferred level of extinction is driven up drastically in the polynomial fit compared to the no-polynomial fit.  This is because the shape of the extinction curve is mildly degenerate with the low-order multiplicative polynomials.  I'd recommend, if you're including multiplicative polynomials in a real fit, to either fix or constrain E(B-V) to alleviate this degeneracy.
